In [71]:
def Strategy_4():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value(亿)": value / 1e8,
            "amount(亿)": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)

    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount(亿)", "value(亿)"]))

Strategy_4()

     code  name    factor  amount(亿)   value(亿)
0  300634  彩讯股份  0.706884  15.272515  21.605400
1  300739  明阳电路  0.611375   9.226874  15.092000
2  603937  丽岛新材  0.594392   7.526991  12.663350
3  300667  必创科技  0.577678   6.012127  10.407400
4  603897  长城科技  0.504811  10.435496  20.672100
5  300675   建科院  0.387464   6.120391  15.796014
6  002806  华锋股份  0.336336   4.841301  14.394224
7  300077  国民技术  0.294075  18.264057  62.106712
8  300514   友讯达  0.279928   2.649522   9.465000
9  603813  原尚股份  0.252288   1.740002   6.896875


In [25]:
import tushare as ts
import os
import datetime
import pymongo
import pandas

In [26]:
client = pymongo.MongoClient("127.0.0.1", 27017)